## Imports & Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import re
import ast
from datasets import load_dataset
from tqdm import tqdm
import helpers
import soft_prompting
import importlib
importlib.reload(helpers)
importlib.reload(soft_prompting)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'mps'

# CONSTANTS
NUM_EPOCHS_FT = 100
NUM_EPOCHS_KD = 100
BATCH_SIZE = 32

## Get Data and preprocess it

In [ ]:
### train data
# data with label and image data
df_train_label = load_dataset('derek-thomas/ScienceQA', split='train')

# df_train_label = df_train_label[df_train_label['solution'] != ''].reset_index()
# df_train_label['input'] = df_train_label.apply(lambda row: helpers.build_prompt(row)[0], axis=1)
# df_train_label[['index', 'input', 'answer', 'explanation', 'image']] = df_train_label[['index', 'input', 'answer', 'solution', 'image']]

# # data from Gemini for KD
# df_train_gemini = pd.read_csv('gemini_1_5_flash_output_train.csv', sep="\t")[['index', 'input', 'answer', 'explanation']]
# df_train_gemini = pd.merge(df_train_gemini, df_train_label[['index', 'image']], on='index')

In [ ]:
### val data
df_val = pd.DataFrame(load_dataset('derek-thomas/ScienceQA', split='validation'))

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-2B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

In [ ]:
# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

In [ ]:
def tokenize_dataset(df, tokenizer, input_column="input"):

    tokenized_data = []
    for _, row in df.iterrows():
        sample = {}
        sample["input_ids"] = tokenizer.encode(row[input_column], return_tensors="pt").squeeze(0)
        
        tokenized_data.append(sample)
    return tokenized_data


In [ ]:
# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": df_train_label[11]['image'],
            },
            {"type": "text", "text": df_train_label[11]['question'] + " " + ' '.join(df_train_label[11]['choices'])},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(device)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


In [ ]:
df_train_label[11]

In [ ]:
df_train_label[11]['image']

In [ ]:
df_train_gemini.head(1)

In [ ]:
model_name = "Qwen2-VL-2B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Datensatz tokenisieren
tokenized_data = tokenize_dataset(df_train_gemini, tokenizer, input_column="input", label_column="answer")

# Dataset erstellen
dataset = SoftPromptingDataset(tokenized_data)

# Zugriff auf ein Beispiel
print(dataset[0])

## Functions for model training

In [ ]:
def train(dataloader_train, dataloader_val):
    train_errors = []
    val_errors = []
    for epoch in tqdm(range(NUM_EPOCHS_FT)):
        error = 0
        num_samples = 0
        for input, answer, explanation, image in dataloader_train:
            pass
            # forward
            # loss = ...
            # loss.backward()
            # optimizer.step()
            # error += loss.item() * X.size(0)
            # num_samples += X.size(0)
        error /= num_samples
        print(f'Error after epoch {epoch}: {error}')
        train_errors.append((epoch, error))
        if epoch % 10:
            val_error = 0
            num_samples
            for input, answer, explanation, image in dataloader_val:
                pass
                # forward
                # loss = ...
                # val_error += loss.item() * X.size(0)
                # num_samples += X.size(0)
            val_error /= num_samples
            print(f'Validation error after epoch {epoch}: {val_error}')
            val_errors.append((epoch, val_error))
    return train_errors_ft, val_error

In [ ]:
def visualize_error(train_errors, val_errors):
    plt.plot(zip(*train_errors), label="Train Error", marker="o", linestyle="-")
    plt.plot(zip(*val_errors), label="Train Error", marker="o", linestyle="-")
    plt.title("Train and Validation Error over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Error")
    plt.show()

## Soft prompting using labels

In [ ]:
# model = None # ToDo
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


model_fine_tuned = soft_prompting.MultimodalSoftPrompting.from_pretrained(model)
# DataLoader for train data
dataset_label_train = SoftPromptingDataset(df_train_label, model_fine_tuned)
dataloader_label_train=DataLoader(dataset_label_train, batch_size=32, shuffle=True)
# DataLoader for val data
dataset_label_val = SoftPromptingDataset(df_val, model_fine_tuned)
dataloader_label_val=DataLoader(dataset_label_val, batch_size=32, shuffle=True)

In [ ]:
train_errors_ft, val_errors_ft = train(dataloader_label_train, dataloader_label_val)

In [ ]:
visualize_error(train_errors_ft, val_errors_ft)

## Knowledge Distillation

In [ ]:
model = None # ToDo
model_knowledge_distillation = soft_prompting.MultimodalSoftPrompting.from_pretrained(model)
# DataLoader for train data
dataset_gemini_train = SoftPromptingDataset(df_train_gemini, model_fine_tuned)
dataloader_gemini_train=DataLoader(dataset_gemini_train, batch_size=32, shuffle=True)
# DataLoader for val data
dataset_gemini_val = SoftPromptingDataset(df_val, model_fine_tuned)
dataloader_gemini_val=DataLoader(dataset_gemini_val, batch_size=32, shuffle=True)

In [ ]:
train_errors_kd, val_errors_kd = train(dataset_gemini_train, dataloader_gemini_train)

In [ ]:
visualize_error(train_errors_kd, val_errors_kd)